# Module 7: Synthetic Data Generation and RAG Evaluation with Ragas

In this notebook, we'll go end-to-end from **generating synthetic evaluation data** to **systematically evaluating and improving a RAG pipeline** — all using [Ragas](https://github.com/explodinggradients/ragas).

The flow is:
1. **Generate** synthetic test data using Ragas' knowledge graph-based approach
2. **Build** a baseline RAG application with LangChain and LangGraph
3. **Evaluate** the RAG application against our synthetic test set using Ragas metrics
4. **Iterate** on the pipeline and measure the impact

> **NOTE:** Ragas is framework-agnostic — while this example uses LangChain/LangGraph, you can use Ragas with any framework (or none at all). Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

## Outline

**Part 1: Synthetic Data Generation**
- Task 1: Dependencies and API Keys
- Task 2: Data Preparation
- Task 3: Knowledge Graph Construction
- Task 4: Generating Synthetic Test Data
- ***❓ Question #1 & Question #2***
- ***🏗️ Activity #1: Custom Query Distribution***

**Part 2: RAG Evaluation with Ragas**
- Task 5: Building a Baseline RAG Application
- Task 6: Evaluating with Ragas
- Task 7: Making Adjustments and Re-Evaluating
- ***❓ Question #3, Question #4, Question #5, & Question #6***
- ***🏗️ Activity #2: Implement a Different Reranking Strategy***

---
# Part 1: Synthetic Data Generation with Ragas

Before we can evaluate a RAG system, we need high-quality test data. Manually creating question-answer pairs is time-consuming and often biased toward simple queries. Ragas solves this by building a **knowledge graph** from your documents and using it to generate diverse, realistic test questions automatically.

We'll use the **Stone Ridge 2025 Investor Letter** and an **Alternative Investments Handbook** as our source documents — maintaining continuity with the investment advisory use case from previous sessions.

## Task 1: Dependencies and API Keys

If you have not already done so, install the required libraries using the uv package manager:
```bash
uv sync
```

We'll need API keys for:
- **OpenAI** — for LLM and embedding models (used in both SDG and RAG evaluation)
- **Cohere** — for reranking in the improved pipeline ([sign up here](https://docs.cohere.com/reference/about))

You have two options for supplying your API keys:
- Use environment variables (copy `.env.sample` to `.env` and fill in your keys)
- Provide them via the prompts below

In [47]:
import nest_asyncio
nest_asyncio.apply()

In [48]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/chris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chris/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [49]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

## Task 2: Data Preparation

We'll prepare our data using two complementary investment-focused sources:
- **Stone Ridge 2025 Investor Letter** — covering Stone Ridge's investment philosophy, Bayesian approach to decision-making, energy investments, reinsurance, and risk management
- **Alternative Investments Handbook** — covering alternative asset classes including real estate, private equity, hedge funds, reinsurance, commodities, and infrastructure

The topical overlap between these documents (particularly around reinsurance, risk premiums, diversification, and alternative investments) helps Ragas build rich cross-document relationships in the knowledge graph.

In [50]:
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader

# Load the Stone Ridge 2025 Investor Letter (PDF)
pdf_loader = PyMuPDFLoader("data/Stone Ridge 2025 Investor Letter.pdf")
pdf_docs = pdf_loader.load()

# Load the Alternative Investments Handbook (text)
txt_loader = TextLoader("data/AlternativeInvestmentsHandbook.txt")
txt_docs = txt_loader.load()

# Combine into a single list
docs = pdf_docs + txt_docs
print(f"Loaded {len(docs)} documents:")
print(f"  - Stone Ridge 2025 Investor Letter: {len(pdf_docs)} pages")
print(f"  - AlternativeInvestmentsHandbook.txt: {len(txt_docs)} document(s)")

Loaded 15 documents:
  - Stone Ridge 2025 Investor Letter: 14 pages
  - AlternativeInvestmentsHandbook.txt: 1 document(s)


## Task 3: Knowledge Graph Construction

Ragas uses a **knowledge graph-based approach** to create synthetic test data. This is powerful because it allows us to create complex, multi-hop queries — not just simple factoid questions. Systems tend to perform well on simple evaluation tasks, so this additional complexity helps us find real weaknesses.

The process works in three stages:
1. **Build the graph** — insert documents as nodes
2. **Apply transformations** — extract headlines, summaries, themes, entities, and embeddings
3. **Create relationships** — use cosine similarity and overlap scores to connect related nodes

Let's start by defining our `generator_llm` and `generator_embeddings`.

In [51]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

### Step 1: Initialize the Knowledge Graph

We create an empty knowledge graph and populate it with our document nodes. Each full document becomes a node of type `DOCUMENT`.

In [52]:
from ragas.testset.graph import KnowledgeGraph, Node, NodeType

kg = KnowledgeGraph()

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 15, relationships: 0)

### Step 2: Apply Transformations

Now we apply the [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms) to enrich our knowledge graph. These transformations:

- **HeadlinesExtractor** — finds the overall headlines for each document
- **SummaryExtractor** — produces summaries of the documents
- **ThemesExtractor** — extracts broad themes
- **EmbeddingExtractor** — creates embeddings for similarity computation
- **NERExtractor** — extracts named entities

These are then used to build relationships between nodes via cosine similarity and overlap scoring.

In [53]:
from ragas.testset.transforms import default_transforms, apply_transforms

transforms = default_transforms(
    documents=docs,
    llm=generator_llm,
    embedding_model=generator_embeddings
)
apply_transforms(kg, transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/14 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/15 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/25 [00:00<?, ?it/s]

Property 'summary' already exists in node '2db385'. Skipping!
Property 'summary' already exists in node '5bb9b5'. Skipping!
Property 'summary' already exists in node 'b61634'. Skipping!
Property 'summary' already exists in node 'e46f0d'. Skipping!
Property 'summary' already exists in node '078d76'. Skipping!
Property 'summary' already exists in node 'd33ea7'. Skipping!
Property 'summary' already exists in node 'ca0164'. Skipping!
Property 'summary' already exists in node '6e2f7d'. Skipping!
Property 'summary' already exists in node '0c8363'. Skipping!
Property 'summary' already exists in node 'cb09ce'. Skipping!
Property 'summary' already exists in node 'e4dabf'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/10 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '5bb9b5'. Skipping!
Property 'summary_embedding' already exists in node '2db385'. Skipping!
Property 'summary_embedding' already exists in node '6e2f7d'. Skipping!
Property 'summary_embedding' already exists in node 'e46f0d'. Skipping!
Property 'summary_embedding' already exists in node 'b61634'. Skipping!
Property 'summary_embedding' already exists in node 'ca0164'. Skipping!
Property 'summary_embedding' already exists in node '0c8363'. Skipping!
Property 'summary_embedding' already exists in node '078d76'. Skipping!
Property 'summary_embedding' already exists in node 'd33ea7'. Skipping!
Property 'summary_embedding' already exists in node 'cb09ce'. Skipping!
Property 'summary_embedding' already exists in node 'e4dabf'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 35, relationships: 303)

### Step 3: Save the Knowledge Graph

Knowledge graphs can be saved and loaded, which is useful for iterating on test generation without rebuilding the graph each time.

In [54]:
kg.save("investment_data_kg.json")

# You can reload it later:
# kg = KnowledgeGraph.load("investment_data_kg.json")

## Task 4: Generating Synthetic Test Data

With our knowledge graph built, we can now generate synthetic test data. Ragas provides several **query synthesizers**, each producing a different type of question:

- **`SingleHopSpecificQuerySynthesizer`** — creates questions answerable from a single chunk of context (e.g., *"What is Stone Ridge's approach to reinsurance investing?"*)
- **`MultiHopAbstractQuerySynthesizer`** — creates questions requiring synthesis across multiple chunks at an abstract level (e.g., *"How do alternative risk premiums relate to portfolio diversification?"*)
- **`MultiHopSpecificQuerySynthesizer`** — creates questions requiring specific details from multiple chunks (e.g., *"How does Stone Ridge's Bayesian philosophy connect to their energy investment strategy?"*)

We define a **query distribution** to control the mix of question types.

In [55]:
from ragas.testset.synthesizers import (
    SingleHopSpecificQuerySynthesizer,
    MultiHopAbstractQuerySynthesizer,
    MultiHopSpecificQuerySynthesizer,
)

query_distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
    (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
    (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

In [56]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    knowledge_graph=kg
)

testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How has the city of Chicago influenced your ca...,[and my career has essentially been a three-de...,My career has essentially been a three-decade ...,single_hop_specifc_query_synthesizer
1,How did Fisher exemplify Bayesian principles i...,"[TODAY’S THE DAY Fancy math aside, the foundat...",Fisher exemplified Bayesian principles by usin...,single_hop_specifc_query_synthesizer
2,What is the recent performance of Stone Ridge ...,[Standardized returns as of most recent quarte...,Stone Ridge Energy Equity Tranches had a stand...,single_hop_specifc_query_synthesizer
3,What information does Chapter 1 of The Alterna...,[The Alternative Investments Handbook A Practi...,Chapter 1 explains that alternative investment...,single_hop_specifc_query_synthesizer
4,What NOI means in real estate?,[PART 2: REAL ESTATE INVESTMENTS Chapter 4: Re...,"NOI stands for Net Operating Income, which is ...",single_hop_specifc_query_synthesizer
5,How do commodities and real assets serve as an...,[<1-hop>\n\nPART 6: COMMODITIES AND REAL ASSET...,Commodities and real assets serve as an inflat...,multi_hop_abstract_query_synthesizer
6,How does the central bank demand for gold rela...,[<1-hop>\n\nPART 6: COMMODITIES AND REAL ASSET...,"The central bank demand for gold, as part of t...",multi_hop_abstract_query_synthesizer
7,How learning from experience and updating beli...,[<1-hop>\n\nand my career has essentially been...,The context explains that Bayesian treasure hu...,multi_hop_abstract_query_synthesizer
8,How do Chapter 1's overview of alternative ass...,[<1-hop>\n\nPART 2: REAL ESTATE INVESTMENTS Ch...,Chapter 1 introduces alternative investments s...,multi_hop_specific_query_synthesizer
9,"In context of Chapter 4 and Chapter 13, how do...",[<1-hop>\n\nPART 6: COMMODITIES AND REAL ASSET...,The context from Chapter 4 discusses real esta...,multi_hop_specific_query_synthesizer


### Abstracted SDG (Shortcut)

The above was the **unrolled** process showing each step. Ragas also provides a one-liner that builds the knowledge graph under the hood and generates the test set in a single call. This is convenient for quick iteration:

In [57]:
# Abstracted approach (for reference):
# generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
# dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

##### ✅ Answer:

*Your answer here*

### ❓ Question #2:

Ragas offers both an "unrolled" (manual) approach and an "abstracted" (automatic) approach to synthetic data generation. What are the trade-offs between these two approaches? When would you choose one over the other?

##### ✅ Answer:

*Your answer here*

### 🏗️ Activity #1: Custom Query Distribution

Modify the `query_distribution` to experiment with different ratios of query types.

**Requirements:**
1. Create a custom query distribution with different weights than the default
2. Generate a new test set using your custom distribution
3. Compare the types of questions generated with the default distribution
4. Explain why you chose the weights you did

In [58]:
### YOUR CODE HERE ###

# Define a custom query distribution with different weights
# Generate a new test set and compare with the default

---
# Part 2: RAG Evaluation with Ragas

Now that we have our synthetic test data, we can use it to **systematically evaluate** a RAG pipeline. The idea is simple:
1. Build a RAG application
2. Run our synthetic queries through it
3. Score the results using Ragas metrics
4. Make changes and measure the impact

This gives us a **data-driven approach** to improving our RAG system, rather than relying on vibes.

## Task 5: Building a Baseline RAG Application

We'll build a deliberately simple (and somewhat bad) RAG pipeline as our **baseline**, so we can clearly see the impact of improvements later.

Our baseline uses:
- Tiny chunks (50 characters) with no overlap
- A small embedding model (`text-embedding-3-small`)
- Only 3 retrieved documents
- A basic prompt

> **NOTE:** We use the same data that our synthetic test set was generated from — this is required because the test questions are specifically designed for this investment data.

### R — Retrieval

First, we chunk our documents and build a vector store.

In [59]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

2045

### ❓ Question #3:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

##### ✅ Answer:

*Your answer here*

In [60]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [61]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="baseline_rag",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="baseline_rag",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_documents)

In [62]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [63]:
def retrieve(state):
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}

### A — Augmented

A simple RAG prompt:

In [64]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful investment advisory assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### G — Generation

We use `gpt-4.1-nano` for generation to avoid using the same model as our judge model.

In [65]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

In [66]:
def generate(state):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
    response = llm.invoke(messages)
    return {"response": response.content}

### Building the RAG Graph with LangGraph

In [67]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
    question: str
    context: List[Document]
    response: str

In [68]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a quick sanity check:

In [69]:
response = graph.invoke({"question": "What is Stone Ridge's investment philosophy?"})
response["response"]

"Stone Ridge's investment philosophy is centered on a relentless focus on growing."

With tiny 50-character chunks and only 3 retrieved documents, the baseline likely struggles to provide good answers about Stone Ridge's investment philosophy. That's intentional — it gives us room to improve!

## Task 6: Evaluating with Ragas

Now we can evaluate our baseline RAG against the synthetic test data we generated in Part 1.

First, we run all the synthetic queries through our RAG pipeline to collect responses and retrieved contexts.

In [70]:
for test_row in testset:
    response = graph.invoke({"question": test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"]
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

Convert to an `EvaluationDataset` for smoother evaluation:

In [71]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(testset.to_pandas())

We select a **judge model** — a separate, capable model that scores the outputs. Using a different model than the generator avoids self-evaluation bias.

In [72]:
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

### Running the Baseline Evaluation

We evaluate across six metrics:
- **Context Recall** — did we retrieve the relevant context?
- **Faithfulness** — is the answer grounded in the retrieved context?
- **Factual Correctness** — is the answer factually correct vs. the reference?
- **Answer Relevancy** — is the answer relevant to the question?
- **Context Entity Recall** — did we capture the key entities from the reference context?
- **Noise Sensitivity** — is the answer affected by irrelevant retrieved content?

In [73]:
from ragas.metrics import (
    LLMContextRecall,
    Faithfulness,
    FactualCorrectness,
    ResponseRelevancy,
    ContextEntityRecall,
    NoiseSensitivity,
)
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

baseline_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        LLMContextRecall(),
        Faithfulness(),
        FactualCorrectness(),
        ResponseRelevancy(),
        ContextEntityRecall(),
        NoiseSensitivity(),
    ],
    llm=evaluator_llm,
    run_config=custom_run_config,
)
baseline_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[30]: AttributeError('StringIO' object has no attribute 'classifications')
Exception raised in Job[37]: AttributeError('StringIO' object has no attribute 'statements')


{'context_recall': 0.1433, 'faithfulness': 0.3754, 'factual_correctness': 0.5409, 'answer_relevancy': 0.6851, 'context_entity_recall': 0.1887, 'noise_sensitivity_relevant': 0.0000}

## Task 7: Making Adjustments and Re-Evaluating

Now that we have a baseline, let's improve the pipeline and measure the impact. We'll make three changes:

1. **Larger chunks** (500 characters with 30 overlap instead of 50 with 0 overlap)
2. **More documents retrieved** (k=20 instead of k=3)
3. **Reranking with Cohere** — retrieves 20 documents, then uses Cohere's reranker to select the top 5

Reranking is a technique that uses a cross-encoder model (slower but more accurate than embedding similarity) on a smaller subset of candidates to improve retrieval precision.

In [74]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
split_documents = text_splitter.split_documents(docs)
print(f"Improved chunking: {len(split_documents)} chunks (vs baseline)")

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(":memory:")
client.create_collection(
    collection_name="improved_rag",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="improved_rag",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_documents)
adjusted_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Improved chunking: 202 chunks (vs baseline)


In [75]:
from langchain_classic.retrievers.contextual_compression import (
    ContextualCompressionRetriever,
)
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
    compressor = CohereRerank(model="rerank-v3.5")
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=adjusted_retriever,
        search_kwargs={"k": 5},
    )
    retrieved_docs = compression_retriever.invoke(state["question"])
    return {"context": retrieved_docs}

In [76]:
from typing import TypedDict, List
from langchain_core.documents import Document

class AdjustedState(TypedDict):
    question: str
    context: List[Document]
    response: str

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
adjusted_graph_builder.add_edge(START, "retrieve_adjusted")
adjusted_graph = adjusted_graph_builder.compile()

Let's verify the improved pipeline works:

In [77]:
response = adjusted_graph.invoke({"question": "How does Stone Ridge approach risk management in their energy investments?"})
response["response"]

'The provided context does not include specific details on how Stone Ridge approaches risk management in their energy investments.'

### Running the Improved Evaluation

Now let's run the same synthetic test set through our improved pipeline and compare.

In [78]:
import time
import copy

rerank_testset = copy.deepcopy(testset)

for test_row in rerank_testset:
    response = adjusted_graph.invoke({"question": test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"]
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
    time.sleep(2)  # To avoid rate limiting

In [46]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(rerank_testset.to_pandas())

rerank_result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[
        LLMContextRecall(),
        Faithfulness(),
        FactualCorrectness(),
        ResponseRelevancy(),
        ContextEntityRecall(),
        NoiseSensitivity(),
    ],
    llm=evaluator_llm,
    run_config=custom_run_config,
)
rerank_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

{'context_recall': 0.6833, 'faithfulness': 0.5825, 'factual_correctness': 0.5364, 'answer_relevancy': 0.7742, 'context_entity_recall': 0.3159, 'noise_sensitivity_relevant': 0.1507}

### ❓ Question #4:

Which system performed better, on what metrics, and why?

##### ✅ Answer:

*Your answer here*

### ❓ Question #5:

What are the benefits and limitations of using synthetic data generation for RAG evaluation? Consider both the practical advantages and potential pitfalls.

##### ✅ Answer:

*Your answer here*

### ❓ Question #6:

If you were building a production investment advisory assistant for Stone Ridge, which Ragas metrics would be most important to optimize for and why? Consider the financial services domain specifically.

##### ✅ Answer:

*Your answer here*

### 🏗️ Activity #2: Implement a Different Reranking Strategy

Experiment with different reranking parameters or strategies to see how they affect the evaluation metrics.

**Requirements:**
1. Modify the `retrieve_adjusted` function to use different parameters (e.g., change `k` values, try different `top_n` for reranking)
2. Or implement a different retrieval enhancement strategy (e.g., hybrid search, query expansion)
3. Run the evaluation and compare results with the baseline and reranking results above
4. Document your findings in the markdown cell below

In [ ]:
### YOUR CODE HERE ###

# Implement your custom retrieval strategy here
# Example: modify retrieve_adjusted with different parameters

def retrieve_custom(state):
    # Your implementation here
    pass

### Activity #2 Findings:

*Document your findings here: What strategy did you try? How did it compare to the baseline and reranking results?*

---
## Summary

In this notebook, we went end-to-end from data generation to evaluation:

1. **Built a knowledge graph** from our investment documents (Stone Ridge 2025 Investor Letter and Alternative Investments Handbook) and used it to understand the structure of our data
2. **Generated synthetic test data** with diverse query types (single-hop, multi-hop abstract, multi-hop specific)
3. **Built a baseline RAG pipeline** with deliberately simple parameters
4. **Evaluated with Ragas** across six metrics to establish a baseline
5. **Improved the pipeline** with larger chunks and Cohere reranking
6. **Re-evaluated** to measure the impact of our changes

### Key Takeaways:

- **Synthetic data generation** is critical for early iteration — it provides high-quality signal without manually creating test data
- **Ragas metrics** give you a multi-dimensional view of RAG quality (retrieval vs. generation vs. faithfulness)
- **Small changes matter** — chunk size, retrieval strategy, and reranking can dramatically affect evaluation scores
- **Always use a different model for judging** than for generating to avoid self-evaluation bias